In [1]:
from typing import List
import json

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator, Required
from langchain_openai import ChatOpenAI
import datetime

import langchain

In [2]:
langchain.debug = True

In [3]:
model = ChatOpenAI(temperature=0)

In [4]:
sap_entry = (
    "SYSTEM_09 went down on Wednesday, 1/24/2024 for an automation error. "
    "Arrived at the tool and found a wafer hanging off of the ETU. "
    "Recovered the wafer and checked wafer grab positions using the FI Wizard. "
    "FI Wizard succeeded with no changes to grab or loading positions. "
    "CE returned the tool to production. "
    "Tool went down again for an automatione error. "
    "Cycled wafers through the system. "
    "While unloading the wafers, automation error occured again. "
    "Recovered the wafers again and checked LLK to ETU handoffs. "
    "Found large shift in grab positions. "
    "Visual inspection of the LLK revealed the O-ring had popped off the PAL base. "
    "Opened the LLK lid, replaced the PAL base with a new unit and closed the LLK lid. "
    "Verifying handoffs between LLK and ETU. "
    "No calibrations needed. "
    "Tool needs to run particle tests before releasing to production. "
    "Tool passed all particle tests and was released to production."
)

In [5]:
print(sap_entry)

SYSTEM_09 went down on Wednesday, 1/24/2024 for an automation error. Arrived at the tool and found a wafer hanging off of the ETU. Recovered the wafer and checked wafer grab positions using the FI Wizard. FI Wizard succeeded with no changes to grab or loading positions. CE returned the tool to production. Tool went down again for an automatione error. Cycled wafers through the system. While unloading the wafers, automation error occured again. Recovered the wafers again and checked LLK to ETU handoffs. Found large shift in grab positions. Visual inspection of the LLK revealed the O-ring had popped off the PAL base. Opened the LLK lid, replaced the PAL base with a new unit and closed the LLK lid. Verifying handoffs between LLK and ETU. No calibrations needed. Tool needs to run particle tests before releasing to production. Tool passed all particle tests and was released to production.


In [21]:
class Action(BaseModel):
    index: int = Field(Required, gt = 0, description = "The order of the action performed.")
    action: str = Field(Required, description = "A specific action done.")
#    resolution: bool = Field(Required, description = "Did the action resolve the root cause? ")
        
class Symptom(BaseModel):
    symptom: str = Field(Required, description = "A symptom of the primary problem statement.")
        
class ToolEvent(BaseModel):
    tool_name: str = Field(Required, description = "The name of the system.")
    start_time: datetime.datetime = Field(Required, description = "When the issue started.")
    problem_statement: str = Field(Required, description= "A high level problem statement describing hte issue.")
    symptoms: List[Symptom] = Field(Required, description = "A list of symptoms associated with the problem.")
    actions: List[Action] = Field(Required, description = "A list os actions performed.")
    root_cause: str = Field(Required, description = "The final root cause of the initial problem statement.")
    resolution: str = Field(Required, description = "What was done to resolve the root cause.")

In [22]:
template = (
    "You are going to be given an entry from a service case submitted by an engineer. "
    "You need to identify specific items from this entry based on the following schema. "
    "Be as concise as possible. "
    "\n{format_instructions}\n{query}\n"
)

In [23]:

parser = PydanticOutputParser(pydantic_object=ToolEvent)

prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query = sap_entry)
output = model.invoke(_input.to_string())
response = parser.parse(output.content)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are going to be given an entry from a service case submitted by an engineer. You need to identify specific items from this entry based on the following schema. Be as concise as possible. \nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"properties\": {\"tool_name\": {\"title\": \"Tool Name\", \"description\": \"The name of the system.\", \"type\": \"string\"}, \"start_time\": {\"title\": \"Start Time\", \"description\": \"When the issue started.\", \

In [24]:
response.dict()

{'tool_name': 'SYSTEM_09',
 'start_time': datetime.datetime(2024, 1, 24, 0, 0, tzinfo=datetime.timezone.utc),
 'problem_statement': 'Automation error',
 'symptoms': [{'symptom': 'Wafer hanging off of the ETU'},
  {'symptom': 'Automation error while unloading wafers'},
  {'symptom': 'Large shift in grab positions'},
  {'symptom': 'O-ring popped off the PAL base'}],
 'actions': [{'index': 1,
   'action': 'Recovered the wafer and checked wafer grab positions using the FI Wizard'},
  {'index': 2, 'action': 'Returned the tool to production'},
  {'index': 3, 'action': 'Cycled wafers through the system'},
  {'index': 4,
   'action': 'Recovered the wafers again and checked LLK to ETU handoffs'},
  {'index': 5,
   'action': 'Opened the LLK lid, replaced the PAL base with a new unit and closed the LLK lid'},
  {'index': 6, 'action': 'Verified handoffs between LLK and ETU'},
  {'index': 7, 'action': 'Performed particle tests'},
  {'index': 8, 'action': 'Released the tool to production'}],
 'root_

In [25]:
print(json.dumps(response.dict(), indent = 4, default=str))

{
    "tool_name": "SYSTEM_09",
    "start_time": "2024-01-24 00:00:00+00:00",
    "problem_statement": "Automation error",
    "symptoms": [
        {
            "symptom": "Wafer hanging off of the ETU"
        },
        {
            "symptom": "Automation error while unloading wafers"
        },
        {
            "symptom": "Large shift in grab positions"
        },
        {
            "symptom": "O-ring popped off the PAL base"
        }
    ],
    "actions": [
        {
            "index": 1,
            "action": "Recovered the wafer and checked wafer grab positions using the FI Wizard"
        },
        {
            "index": 2,
            "action": "Returned the tool to production"
        },
        {
            "index": 3,
            "action": "Cycled wafers through the system"
        },
        {
            "index": 4,
            "action": "Recovered the wafers again and checked LLK to ETU handoffs"
        },
        {
            "index": 5,
            "a